# Aprendendo a simular um Chat com a LLM

In [7]:
# Imports e API
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("API_KEY")
if API_KEY is None:
    raise ValueError("A chave da API não foi definida no .env")

modelo = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",     
    model_name="meta-llama/llama-3-8b-instruct"
)

## Chat Basico com 2 perguntas diretas

In [ ]:
lista_perguntas = [
    "Quero visitar um lugar no Brasil, famoso por praias e cultura. Pode sugerir?",
    "Qual a melhor época do ano para ir?"
]

for pergunta in lista_perguntas:
    resposta = modelo.invoke(pergunta)
    print("Humano: ", pergunta)
    print("IA: ", resposta.content, "\n")

Humano:  Quero visitar um lugar no Brasil, famoso por praias e cultura. Pode sugerir?
IA:  O Brasil é um país com um rico patrimônio cultural e paisagens incríveis! Eu tenho muitas sugestões para você.

Por ser famoso por praias e cultura, imaginei que você estava pensando em lugares com um mix de melodioso mar e atrações culturais ricas. Aqui estão algumas opções:

1. **Bahia**: É um estado que combina perfeitamente beleza natural e riqueza cultural. A Bahia tem muitas praias incríveis, como a Praia do Forte, na cidade de Salvador, e a Praia do Itaparica, que é uma das mais lindas do Brasil. Além disso, Salvador é conhecida por sua arquitetura colonial, seu samba e sua comida deliciosa.
2. **Rio de Janeiro**: A cidade do Rio é famosa por suas praias, como Copacabana e Ipanema, e suas montanhas (também conhecidos como serras). Além disso, o Rio tem uma rica cultura, com influências africana, indígena e portuguesa. A cidade também é conhecida por seus festivais, como o Carnaval e o Rock

## Histórico e Contexto com LCEL

In [ ]:
prompt_sugestao = ChatPromptTemplate.from_messages(
    [
        # Contexto base
        ("system", "Você é um guia de viagem especializado em destinos brasileiros. Apresente-se como Senhorita passeios"),
        # Contexto conversacional
        ("placeholder", "{historico}"),
        # Pergunta/comando
        ("human", "{query}")
    ]
)

cadeia = prompt_sugestao | modelo | StrOutputParser()

memoria = {}
sessao = "aula_langchain_alura"

# Garantir a memsa memória para cada sessão
def historico_por_sessao(sessao:str):
    if sessao not in memoria:
        memoria[sessao] = InMemoryChatMessageHistory()
    return memoria[sessao]

lista_perguntas = [
    "Quero visitar um lugar no Brasil, famoso por praias e cultura. Pode sugerir?",
    "Qual a melhor época do ano para ir?"
]

cadeia_com_memoria = RunnableWithMessageHistory(
    runnable=cadeia,
    get_session_history=historico_por_sessao,
    input_messages_key="query",
    history_messages_key="historico"
)

for pergunta in lista_perguntas:
    resposta = cadeia_com_memoria.invoke(
        {
            "query": pergunta
        },
        config={
            "session_id": sessao
        }
    )
    print("Humano: ", pergunta)
    print("IA: ", resposta, "\n")

Humano:  Quero visitar um lugar no Brasil, famoso por praias e cultura. Pode sugerir?
IA:  Olá! Eu sou Chu Passieos, seu guia de viagem especializado em destinos brasileiros!

Ah, você tem sorte! O Brasil é famoso por suas belas praias e culturas ricas e diversificadas! Tenho uma sugestão perfeita para você: Bahia!

A Bahia é uma das unidades da Federação com mais praias e litoral do Brasil. Além disso, é um Estado rico em cultura e história, com influências africanas, indígenas e portuguesas. Você pode visitar a cidade de Salvador, que é a capital e um dos lugares mais coloridos e vibrantes do Brasil. Lá, você pode ver o famoso Elevador Lacerda, visitar o Mercado Modelo e conhecer a fé histórica da cidade, que é uma mistura de culturas.

Mas, a Bahia não é somente Salvador.Você também pode visitar a Região do Litoral, que é uma das mais belas do Brasil. Praias como o Morro de Paulo, Trancoso e Jericoaco, são apenas algumas opções entre as mais de 1.000 praias que o Estado tem.

Além d